In [1]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip install pyvirtualdisplay
!pip install pyglet==1.5.1

In [2]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [3]:
%%capture
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt
!pip install swig
!pip install gym[box2d]
!pip install stable_baselines3

In [4]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gym
import gym_pygame

# Hugging Face Hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.
import imageio

# Stable Baselines
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
print(device)

cuda:0


In [7]:
# Make Vectorized Environment
vec_env = make_vec_env("CarRacing-v2", n_envs=4, env_kwargs={'continuous': False})
vec_env = VecFrameStack(vec_env, 4)

In [8]:
model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=10000, progress_bar=True)

model.save("ppo_carracing")

Using cuda device
Wrapping the env in a VecTransposeImage.


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -53.8    |
| time/              |          |
|    fps             | 54       |
|    iterations      | 1        |
|    time_elapsed    | 149      |
|    total_timesteps | 8192     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1e+03      |
|    ep_rew_mean          | -48.8      |
| time/                   |            |
|    fps                  | 51         |
|    iterations           | 2          |
|    time_elapsed         | 319        |
|    total_timesteps      | 16384      |
| train/                  |            |
|    approx_kl            | 0.01360761 |
|    clip_fraction        | 0.118      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.6       |
|    explained_variance   | -0.0161    |
|    learning_rate        | 0.0003     |
|   

In [9]:
# Evaluation
model = PPO.load("ppo_carracing")
mean_reward, std_reward = evaluate_policy(model, vec_env, n_eval_episodes=10)

print(mean_reward, std_reward)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


-44.6343581 40.21446100686819


In [10]:
def record_video(env, policy, out_directory, fps=30):
  """
  Generate a replay video of the agent
  :param env
  :param Qtable: Qtable of our agent
  :param out_directory
  :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
  """
  images = []
  done = [False]
  x = 0
  state = vec_env.reset()
  img = vec_env.render(mode='rgb_array')
  images.append(img)
  while not any(done) and x <= 10000:
    # Take the action (index) that have the maximum expected future reward given that state
    action, _ = policy.predict(state)
    state, reward, done, info = vec_env.step(action) # We directly put next_state = state for recording logic
    img = vec_env.render(mode='rgb_array')
    images.append(img)
    x += 1
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [11]:
vec_env = make_vec_env("CarRacing-v2", n_envs=1, env_kwargs={'continuous': False})
vec_env = VecFrameStack(vec_env, 4)

record_video(vec_env, model, './replay.mp4')